In [36]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_curve, auc, confusion_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
from lifelines import KaplanMeierFitter

In [37]:
# 0. Setup
outdir = r"C:\Users\tm1621\Desktop\Protein Project"
os.makedirs(outdir, exist_ok=True)
r_dir  = r"C:\Users\tm1621\Desktop\Protein Project\R Results"

# 1. LOAD DATA FILES
maf_file = os.path.join(outdir, "cohortMAF.2025-05-25.maf")
clinical_file = r"C:\Users\tm1621\Desktop\Protein Project\clinical.cohort.2025-05-25\clinical.tsv"

maf = pd.read_csv(maf_file, sep="\t", comment="#", low_memory=False)
clinical = pd.read_csv(clinical_file, sep="\t", low_memory=False)

In [38]:
# 2 & 3. FEATURE AGGREGATION & CLEANING
drivers = ["KRAS", "TP53", "APC", "SMAD4"]
vc_types = ["Missense_Mutation", "Nonsense_Mutation", "Frame_Shift_Del", "Frame_Shift_Ins"]

# a) driver flags like data.table::dcast
maf_drivers = maf[
    maf["Hugo_Symbol"].isin(drivers) &
    maf["Variant_Classification"].isin(vc_types)
]
flags = (
    maf_drivers
    .groupby(["Tumor_Sample_Barcode", "Hugo_Symbol"])
    .size()
    .unstack(fill_value=0)                 # 0 = no hits
    .map(lambda x: 1 if x > 0 else 0) # convert to binary
    .reset_index()
)

# b) tumor mutational burden
tmb = (
    maf[maf["Variant_Classification"].isin(vc_types)]
    .groupby("Tumor_Sample_Barcode")
    .size()
    .reset_index(name="Mutation_Count")
)

# c) Merge and join clinical
flags["Sample_ID"] = flags["Tumor_Sample_Barcode"].str[:12]
tmb["Sample_ID"] = tmb["Tumor_Sample_Barcode"].str[:12]
feature_table = (
    flags
    .merge(tmb, on=["Tumor_Sample_Barcode", "Sample_ID"], how="outer")
    .merge(clinical,
           left_on="Sample_ID",
           right_on="cases.submitter_id",
           how="left")
)

feature_table['Age'] = pd.to_numeric(
    feature_table['demographic.age_at_index'],
    errors='coerce'
)

# b) Pull out the exact same columns into model_data
model_data = feature_table[[
    'KRAS','TP53','APC','SMAD4','Mutation_Count',
    'Age',
    'diagnoses.ajcc_pathologic_stage',
    'demographic.vital_status',
    'diagnoses.days_to_last_follow_up'
]].copy()

# c) Rename to match R
model_data.columns = [
    'KRAS','TP53','APC','SMAD4','Mutation_Count',
    'Age','Stage','Outcome','FollowUp'
]

# d) Cast Stage and Outcome as categorical (factor in R)
model_data['Stage']   = model_data['Stage'].astype('category')
model_data['Outcome'] = model_data['Outcome'].astype('category')


# e) Drop any row with an actual NA in *any* of these columns,
#    exactly like R’s na.omit(model_data)
print("Rows before dropna:", model_data.shape[0])
model_data = model_data.dropna()
print("Rows after  dropna:", model_data.shape[0])

Rows before dropna: 2750
Rows after  dropna: 2106


In [39]:
# Convert FollowUp to numeric *after* the initial dropna
model_data['FollowUp'] = pd.to_numeric(
    model_data['FollowUp'], errors='coerce'
)
print("Rows after  dropna:", model_data.shape[0])

Rows after  dropna: 2106


In [40]:
df_r = pd.read_csv(r"C:\Users\tm1621\Desktop\Protein Project\R Results\model_data_cleaned.csv")
df_py  = pd.read_csv(r"C:\Users\tm1621\Desktop\Protein Project\model_data_cleaned.csv")
print(df_py.shape, df_r.shape)

(2106, 9) (2106, 9)


In [41]:
model_data['Outcome_num'] = model_data['Outcome'].map({'Dead':1,'Alive':0})
fit_full_py = smf.glm(
    formula='Outcome_num ~ KRAS + TP53 + APC + SMAD4 + Mutation_Count + Age + C(Stage)',
    data=model_data,
    family=sm.families.Binomial()
).fit()
pd.DataFrame({'coef': fit_full_py.params}).to_csv(
    os.path.join(outdir, "full_coefs_PY.csv")
)

### **~~~~~~~~~~~~~~~~~~ should be fine up to here ~~~~~~~~~~~~~~~~~~**

In [42]:
levels = model_data['Stage'].astype('category').cat.categories
model_data['Stage'] = pd.Categorical(
    model_data['Stage'],
    categories=levels,
    ordered=True
)
# 2) Train/test split (same 70% R used)
n       = len(model_data)
n_train = int(0.7 * n)
shuf    = model_data.sample(frac=1, random_state=1234).reset_index(drop=True)
train_py = shuf.iloc[:n_train].copy()
test_py  = shuf.iloc[n_train:].copy()

train_py['Outcome_num'] = train_py['Outcome'].map({'Dead':1,'Alive':0})

In [ ]:
formula = (
    'Outcome_num ~ KRAS + TP53 + APC + SMAD4 + '
    'Mutation_Count + Age + C(Stage)'
)
logit_py = smf.glm(
    formula=formula,
    data=train_py,
    family=sm.families.Binomial()
).fit()

print(logit_py.summary())

                          Generalized Linear Model Regression Results                           
Dep. Variable:     ['Outcome_num[0]', 'Outcome_num[1]']   No. Observations:                 1474
Model:                                              GLM   Df Residuals:                     1454
Model Family:                                  Binomial   Df Model:                           19
Link Function:                                    Logit   Scale:                          1.0000
Method:                                            IRLS   Log-Likelihood:                -713.62
Date:                                  Wed, 16 Jul 2025   Deviance:                       1427.2
Time:                                          17:14:08   Pearson chi2:                 1.45e+03
No. Iterations:                                      22   Pseudo R-squ. (CS):             0.1467
Covariance Type:                              nonrobust                                         
                             c

In [44]:
mapping = {"Intercept": "(Intercept)"}
for lvl in levels[1:]:   # skip levels[0], the reference
    py_key = f'C(Stage)[T.{lvl}]'
    r_key  = f"Stage{lvl}"  # e.g. "StageStage IA"
    mapping[py_key] = r_key

coef_df = pd.DataFrame({
    "variable": [mapping.get(k, k) for k in logit_py.params.index],
    "coef":     logit_py.params.values
})
coef_df.to_csv(os.path.join(outdir, "Model_coefficients_for_EMR_PY.csv"), index=False)

# export ORs
conf = logit_py.conf_int()
or_df = pd.DataFrame({
    "variable": [mapping.get(k, k) for k in logit_py.params.index],
    "OR":        np.exp(logit_py.params),
    "CI_lower":  np.exp(conf[0]),
    "CI_upper":  np.exp(conf[1])
})
or_df.to_csv(os.path.join(outdir, "Logistic_ORs_PY.csv"), index=False)

c:\Users\tm1621\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [45]:
r_path  = r"C:\Users\tm1621\Desktop\Protein Project\R Results\Model_coefficients_for_EMR.csv"
py_coef = pd.DataFrame({
    "coef_py": logit_py.params
})
# Load R’s β‐coefficients (log‐odds)
r_coef = pd.read_csv(r_path, index_col="variable")

# Join & compare
cmp = r_coef.join(py_coef, how="inner")
cmp["diff"] = cmp["coef_py"] - cmp["coef"]
print(cmp[["coef","coef_py","diff"]])

                    coef   coef_py      diff
KRAS           -0.579861  0.424172  1.004033
TP53            0.450985 -0.455892 -0.906877
APC             0.417470 -0.471612 -0.889082
SMAD4           0.465998 -0.420641 -0.886639
Mutation_Count  0.000144 -0.000201 -0.000345
Age             0.030341 -0.024643 -0.054984


In [46]:
print("Python Stage categories:", train_py['Stage'].cat.categories.tolist())
print("Python Stage value counts:\n", train_py['Stage'].value_counts(dropna=False))

Python Stage categories: ["'--", 'Stage I', 'Stage IA', 'Stage II', 'Stage IIA', 'Stage IIB', 'Stage IIC', 'Stage III', 'Stage IIIA', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage IVA', 'Stage IVB']
Python Stage value counts:
 Stage
'--           389
Stage IIIB    204
Stage IIA     196
Stage IV      195
Stage I       128
Stage IIIC    123
Stage IVA      62
Stage III      48
Stage II       41
Stage IIIA     36
Stage IIB      28
Stage IVB      18
Stage IA        3
Stage IIC       3
Name: count, dtype: int64
